In [1]:
import strawberryfields as sf
from strawberryfields.ops import *
# from strawberryfields.utils import scale
from numpy import pi, sqrt
import math
import random
import numpy as np

from matplotlib import pyplot as plt



In [2]:
# eng, q = sf.Engine(1)
prog = sf.Program(1)
# eng = sf.Engine('fock',cutoff_dim=10)
# eng = sf.Engine("fock", backend_options={"cutoff_dim":10})
eng = sf.RemoteEngine("simulon_gaussian")



iterations = 45
shots = 10
parabolic_min = 10
possible_parameters = [0.1, 0.3, 0.5, 0.7, 0.9, 1.1]
testing_trials = 100
optimal_value = math.inf
simulation = []


alpha_param = [0.5, 0.5, 0.5]
beta_param = [0.5, 0.5, 0.5]



In [3]:

prog = sf.Program(1)

with prog.context as q:
    Squeezed(-0.5, 0) | q[0]
    Zgate(parabolic_min*4*beta_param[0]) | q[0]
    # Pgate(-4*beta_param[0]) | q[0]
    Rgate(-1*alpha_param[0]) | q[0]
    Zgate(parabolic_min*4*beta_param[1]) | q[0]
    # Pgate(-4*beta_param[1]) | q[0]
    Rgate(-1*alpha_param[1]) | q[0]
    Zgate(parabolic_min*4*beta_param[2]) | q[0]
    # Pgate(-4*beta_param[2]) | q[0]
    Rgate(-1*alpha_param[2]) | q[0]

# Run on local simulator
# eng = sf.Engine(backend="fock", cutoff_dim=5)
# result = eng.run(prog, cutoff_dim=10)
result = eng.run(prog, shots=1)
state = result.state

# Define range of x and p
x_vals = np.linspace(-5, 5, 200)
p_vals = np.linspace(-5, 5, 200)

# Get probabilities
prob_x = state.quad_prob(0, x_vals, basis="x")
prob_p = state.quad_prob(0, p_vals, basis="p")

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(x_vals, prob_x, label="X quadrature")
plt.title("X Quadrature Probability")
plt.xlabel("x")
plt.ylabel("Probability")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(p_vals, prob_p, color="orange", label="P quadrature")
plt.title("P Quadrature Probability")
plt.xlabel("p")
plt.ylabel("Probability")
plt.grid(True)

plt.tight_layout()
plt.show()



2025-06-24 11:22:12,447 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-06-24 11:22:15,498 - INFO - The remote job 94a1eef8-3c21-4371-b1ed-ac8ab3a7464c has been completed.


AttributeError: 'NoneType' object has no attribute 'quad_prob'

In [5]:
print(result)

<Result: shots=0, num_modes=0, contains state=False>


In [ ]:
#Calculating loss

def function_optimize(x, parabolic_min):
    y = (x - parabolic_min)**2
    return y



In [ ]:
#Run the photonic quantum circuit

def run_circuit(alpha_param, beta_param, parabolic_min):

    prog = sf.Program(1)

    # with eng:
    with prog.context as q:

        Squeezed(-0.5,0) | q[0]

        Zgate(parabolic_min*4*beta_param[0]) | q[0]
        Pgate(-4*beta_param[0]) | q[0]
        Rgate(-1*alpha_param[0]) | q[0]
        Zgate(parabolic_min*4*beta_param[1]) | q[0]
        Pgate(-4*beta_param[1]) | q[0]
        Rgate(-1*alpha_param[1]) | q[0]
        Zgate(parabolic_min*4*beta_param[2]) | q[0]
        Pgate(-4*beta_param[2]) | q[0]
        Rgate(-1*alpha_param[2]) | q[0]

        MeasureX | q[0]

        state = eng.run(prog, cutoff_dim=5)

    return q[0].val



In [ ]:
#Search for the optimal value

op = 0

alpha_param = [0.5, 0.5, 0.5]
beta_param = [0.5, 0.5, 0.5]

for i in range(0, iterations):
    result = 0
    grid = []
    for h in range(0, shots):
        hello = run_circuit(alpha_param, beta_param, parabolic_min)
        result = result+hello
        grid.append(hello)
    result = result/shots
    calculation = function_optimize(result, parabolic_min)

    yer_a = alpha_param
    yer_b = beta_param

    if (calculation < optimal_value):
        simulation = grid
        optimal_value = calculation
        the_x_measurement = result
        op = [[alpha_param[0], alpha_param[1], alpha_param[2]], [beta_param[0], beta_param[1], beta_param[2]]]

    alpha_param[0] = possible_parameters[random.randint(0, len(possible_parameters)-1)]
    alpha_param[1] = possible_parameters[random.randint(0, len(possible_parameters)-1)]
    alpha_param[2] = possible_parameters[random.randint(0, len(possible_parameters)-1)]

    beta_param[0] = possible_parameters[random.randint(0, len(possible_parameters)-1)]
    beta_param[1] = possible_parameters[random.randint(0, len(possible_parameters)-1)]
    beta_param[2] = possible_parameters[random.randint(0, len(possible_parameters)-1)]


optimal_parameters = op


x_arr = range(0, testing_trials)
y_arr = []

for i in range(0, testing_trials):

    state = run_circuit(optimal_parameters[0], optimal_parameters[1], parabolic_min)
    y_arr.append(state)

print(y_arr)

[array([9.08319969]), array([9.2528578]), array([9.88208275]), array([9.53255929]), array([9.35851804]), array([9.77843099]), array([9.11970645]), array([9.29762336]), array([9.35000506]), array([9.15215515]), array([9.70096993]), array([9.48673216]), array([9.3905478]), array([9.64843308]), array([9.06176826]), array([9.71188918]), array([9.02572207]), array([9.26750098]), array([9.27324394]), array([9.11846597]), array([9.57813161]), array([9.42052365]), array([9.24956251]), array([9.39129207]), array([9.61852709]), array([8.65362994]), array([9.57566121]), array([9.40468347]), array([9.24130666]), array([9.76703626]), array([9.22959661]), array([9.62663638]), array([9.24931217]), array([9.28639126]), array([9.08960181]), array([9.24777504]), array([9.77912325]), array([9.88579171]), array([9.08704266]), array([9.18592823]), array([9.83398144]), array([9.64354149]), array([9.7241117]), array([9.37855096]), array([9.52975966]), array([9.59239752]), array([9.25156394]), array([9.445016